# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [142]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Users\drggf\Desktop\Data Engineering Nanodegree\Data-Modeling-with-Apache-Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], \
                         row[8], row[12], row[13], row[16]))


In [94]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [137]:
# Make a connection to a Cassandra instance on my local machine on 127.0.0.1
# where I have a Cassandra database running

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [134]:
# Drop a Keyspace
# try:
#     session.execute("""
#     DROP KEYSPACE sparkify 
#     """
# )

# except Exception as e:
#     print(e)

In [138]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [139]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [140]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4


query = "CREATE TABLE IF NOT EXISTS session_history "
query = query + "(session_id int, item_in_session int, artist text, song_title text, \
                song_length double, PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [151]:
# We have provided part of the code to set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_history (session_id, item_in_session, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [152]:
## Validate session history query
# query = "SELECT artist, song_title, song_length FROM session_history WHERE session_id = 338 and item_in_session = 4"
query = "SELECT artist, song_title, song_length FROM session_history WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [ ]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_history "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, \
    first_name text, last_name text, PRIMARY KEY (user_id, session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [37]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_history (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "(%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[11], line[9], line[4], line[1], line[10], line[2], line[5]))

246.30812
144.03873
177.18812
380.42077
440.2673
241.3971
160.15628
214.93506
200.72444
233.7171
245.36771
288.9922
216.42404
221.33506
178.02404
218.46159
190.40608
278.88281
189.6224
175.43791
242.59873
254.48444
224.67873
200.202
179.66975
208.24771
246.59546
251.42812
265.92608
286.17098
132.85832
278.04689
202.29179
276.29669
177.162
264.98567
105.482
218.61832
221.1522
155.66322
319.32036
127.73832
167.83628
294.76526
245.36771
317.98812
234.84036
319.63383
330.16118
368.97914
312.29342
196.33587
326.32118
237.92281
207.38567
219.76771
293.32853
134.47791
261.40689
375.19628
246.49098
188.26404
238.52363
224.65261
168.6722
179.40853
198.97424
357.66812
182.88281
219.08853
205.66159
300.19873
181.08036
310.15138
219.89832
285.43955
198.84363
229.3024
242.99057
221.85751
252.21179
409.86077
256.15628
295.33995
249.91302
212.21832
122.04363
203.28444
222.61506
129.67138
239.22893
300.19873
191.13751
206.34077
169.84771
272.92689
319.73832
229.77261
321.30567
221.3873
313.86077
229.5

In [ ]:
# Validate user history query
query = "SELECT item_in_session, artist, song_title, first_name, last_name FROM user_history \
         WHERE user_id = 10 AND session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.item_in_session, row.artist, row.song_title, row.first_name, row.last_name)

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [91]:
## TO-DO: Drop the table before closing out the sessions

query = "drop table session_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="Table 'sparkify.session_history' doesn't exist"


### Close the session and cluster connection¶

In [92]:
session.shutdown()
cluster.shutdown()